In [19]:
import os
import sys

import warnings
warnings.filterwarnings("ignore")

os.environ['environment'] = 'interactive'
os.environ['UTILS_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/python-utils-ak'
os.environ['PROJECT_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat' 
os.environ['SQLITE_PATH'] = '/Users/arsenijkadaner/Yandex.Disk.localized/master/code/git/2020.10-umalat/umalat/data.sqlite' 
sys.path.append(os.environ['UTILS_PATH'])
sys.path.append(os.environ['PROJECT_PATH'])
from app.interactive_imports import *

In [116]:
from utils_ak.block_tree import *
from utils_ak.iteration import *
from utils_ak.builtin import *

from app.schedule_maker.models import *

In [117]:
boiling_model = cast_model(MascarponeBoiling, 16)
boiling_model

<MascarponeBoiling 16>

In [121]:
sku = cast_model(MascarponeSKU, 82)
sku

<MascarponeSKU 82>

In [126]:
values = [[sku, 10]]
boiling_group_df = pd.DataFrame(values, columns=['sku', 'kg'])
boiling_group_df

,sku,kg
0,<MascarponeSKU 82>,10


In [131]:
def make_mascorpone_boiling(boiling_group_df):
    maker, make = init_block_maker("boiling", boiling_model=boiling_model)
    bt = boiling_model.boiling_technology

    with make('boiling_process'):
        make("pouring", size=(bt.heating_time // 5, 0))
        make("heating", size=(bt.heating_time // 5, 0))
        make("waiting", size=[0, 0])
        make("adding_lactic_acid", size=(bt.heating_time // 5, 0))
        make("separation", size=(bt.heating_time // 5, 0))
    with make('packing_process', x=(maker.root['boiling_process'].x[0], 0)):
        make("N", size=(2, 0))
        make("P", size=(2, 0))

        packing_start = maker.root["packing_process"]["P"].props.relative_props['x'][0] + 1

        packing_time = sum(
            [
                row["kg"] / row["sku"].packing_speed * 60
                for i, row in boiling_group_df.iterrows()
            ]
        )
        packing_time = int(custom_round(packing_time, 5, "ceil"))

        make('packing', x=(packing_start, 0), size=(packing_time//5, 0), push_func=add_push)
    return maker.root

In [155]:
make_mascorpone_boiling(boiling_group_df)

=========================== boiling (0, 27] x (0, 0]
============ boiling_process (0, 12] x (0, 0]
=== pouring (0, 3] x (0, 0]
   === heating (3, 6] x (0, 0]
      === adding_lactic_acid (6, 9] x (0, 0]
         === separation (9, 12] x (0, 0]
            =============== packing_process (12, 27] x (0, 0]
            == N (12, 14] x (0, 0]
              == P (14, 16] x (0, 0]
               ============ packing (15, 27] x (0, 0]

In [165]:
b1, b2 = make_mascorpone_boiling(boiling_group_df), make_mascorpone_boiling(boiling_group_df)

In [169]:
def make_mascarpone_boiling_group(boiling_group_df1, boiling_group_df2):
    validator = ClassValidator(window=3)

    def validate(b1, b2):
        validate_disjoint_by_axis(b1['packing_process']['packing'], b2['packing_process']['packing'])
        # just in case, not needed in reality
        validate_disjoint_by_axis(b1['boiling_process']['separation'], b2['boiling_process']['separation']) 
        assert b2['boiling_process']['pouring'].x[0] >= b1['boiling_process']['pouring'].y[0]

    validator.add("boiling", "boiling", validate)
    
    maker, make = init_block_maker("boiling_group", boiling_model=boiling_model)
    
    b1 = make_mascorpone_boiling(boiling_group_df1)
    b2 = make_mascorpone_boiling(boiling_group_df2)
    for b in [b1, b2]:
        push(
                maker.root,
                b,
                push_func=AxisPusher(start_from="last_beg"),
                validator=validator,

            )
    # fix waiting time 
    waiting_size = b2['boiling_process']['pouring'].x[0] - b1['boiling_process']['pouring'].y[0]
    for key in ['pouring', 'heating', 'waiting']:
        b = b2['boiling_process'][key]
        b.props.update(x=(b.props['x_rel'][0] - waiting_size, 0))
    b2['boiling_process']['waiting'].props.update(size=(waiting_size, 0))
    return maker.root

In [170]:
make_mascarpone_boiling_group(boiling_group_df, boiling_group_df)

2021-03-10 21:32:34.521 | DEBUG    | utils_ak.block_tree.pushers.iterative:update:106 - Disposition


======================================= boiling_group (0, 39] x (0, 0]
=========================== boiling (0, 27] x (0, 0]
============ boiling_process (0, 12] x (0, 0]
=== pouring (0, 3] x (0, 0]
   === heating (3, 6] x (0, 0]
      === adding_lactic_acid (6, 9] x (0, 0]
         === separation (9, 12] x (0, 0]
            =============== packing_process (12, 27] x (0, 0]
            == N (12, 14] x (0, 0]
              == P (14, 16] x (0, 0]
               ============ packing (15, 27] x (0, 0]
            =========================== boiling (12, 39] x (0, 0]
            ============ boiling_process (12, 24] x (0, 0]
   === pouring (3, 6] x (0, 0]
      === heating (6, 9] x (0, 0]
         ========= waiting (9, 18] x (0, 0]
                  === adding_lactic_acid (18, 21] x (0, 0]
                     === separation (21, 24] x (0, 0]
                        =============== packing_process (24, 39] x (0, 0]
                        == N (24, 26] x (0, 0]
                          == 